# Step 1 - 6
### Here is a notebook for [the MedExtractionPipeline](./MedExtractionPipeline.ipynb)

# Step 7: Processing Context

The template of this notebook is largely based on the MultiSentenceDocuments.ipynb example notebook by pyContextNLP

In [1]:
import sys
sys.path.append('../') 
sys.path.append('modules') # Append Map with modules to the source tree
import numpy as np
import pandas as pd
import re
from textblob import TextBlob
#import EMR_functions as em

import modules.pyConTextNLP2.pyConTextNLP.pyConText as pyConText

import modules.pyConTextNLP2 #import itemData
#from textblob import TextBlob
import modules.pyConTextNLP2.pyConTextNLP.display.html as html
from IPython.display import display, HTML

def read_csv(f_name):
    return pd.read_csv(f_name, sep="|")

## Open Dataset

In [2]:
df_explore = pd.read_csv(r'output_files/DF_explore.csv', index_col=0, sep="|", na_values=['NA'])
print(len(df_explore))


FileNotFoundError: File b'output_files/DF_explore.csv' does not exist

In [3]:
df_annotated = df_explore.sample(frac=0.5, replace=True, random_state=777)
pc_df = em.Processing(df_annotated)
pc_df.getDictCutOff()
df_corrected, l_typo_found = pc_df.typoProcessing('XANTWOORD')
df_corrected.to_csv(r'output_files/DF_proc_annotated.csv', sep='|', index=True)
df_corrected['XANTWOORD'].head()

NameError: name 'df_explore' is not defined

In [62]:
df_example = pd.read_csv(r'ExampleDataLong.csv', sep=",", na_values=['NA']) #  index_col=0
#pc_df = em.Processing(df_corrected)
#pc_df.getDF()['XANTWOORD'].head()
df_example.head(20)

,PATNR,drug,DATUM,actie,ToDo,drug_answer,Opmerking
0,1,MTX,2012-05-12,Start,NaN,NaN,NaN
1,1,LEF,2012-05-13,Start,NaN,NaN,NaN
2,1,None,2012-05-14,NaN,Continue,MTX& LEF,NaN
3,1,MTX,2012-05-17,Stop,NaN,NaN,NaN
4,1,LEF,2012-05-18,NaN,Continue,NaN,NaN
5,1,SASP,2012-05-19,Start,NaN,NaN,NaN
6,1,MTX,2012-05-21,Start,NaN,Trigger moet afgaan,NaN
7,1,SASP,2012-05-26,Start,NaN,NaN,NaN
8,1,MTX,2012-06-23,Stop,NaN,NaN,NaN
9,1,MTX,2013-06-21,Stop,NaN,NaN,NaN


In [63]:
d_trajectories = {}
d_cur_drugs = {}
new_df = pd.DataFrame(columns=['PATNR', 'drug','DATUM', 'actie', 'trigger'])

for index, row in df_example.head(11).iterrows():
    key = row['drug'] + str(index)
    if (any(row['drug'] in s for s in d_cur_drugs.keys()) == False): 
        if row['actie'] == 'Stop':
            print('EARLIER STOP!!')
            last_match = [s for s in d_trajectories.keys() if row['drug'] in s][-1]
            print(last_match)
            d_trajectories[key] = [row['drug'], d_trajectories[last_match][1], row['DATUM']]
            new_df = new_df.append({'PATNR' : row['PATNR'], 'drug': row['drug'],'DATUM': row['DATUM'], 'actie': row['actie'], 'trigger' : 'CONFLICT-> is al eerder geeindigd'}, ignore_index=True)
        elif row['actie'] == 'Start':
            d_trajectories[key] = [row['drug'], row['DATUM'], None]
            d_cur_drugs[row['drug']] = key
            new_df = new_df.append({'PATNR' : row['PATNR'], 'drug': row['drug'],'DATUM': row['DATUM'], 'actie': row['actie'], 'trigger' : ''}, ignore_index=True)
        print(d_cur_drugs)
    else :
        
        if row['actie'] == 'Stop': # niet eerder gestopt!
            l_val = d_trajectories[d_cur_drugs[row['drug']]]
            d_trajectories[d_cur_drugs[row['drug']]] = [row['drug'], l_val[1], row['DATUM']]
            del d_cur_drugs [row['drug']]
            new_df = new_df.append({'PATNR' : row['PATNR'], 'drug': row['drug'],'DATUM': row['DATUM'], 'actie': row['actie'], 'trigger' : ''}, ignore_index=True)
        elif row['actie'] == 'Start': # al eerder gestart
            print('Double Start!!')
            d_trajectories[key] = [row['drug'], row['DATUM'], None]
            new_df = new_df.append({'PATNR' : row['PATNR'], 'drug': row['drug'],'DATUM': row['DATUM'], 'actie': row['actie'], 'trigger' : 'CONFLICT-> is al eerder gestart'}, ignore_index=True)
        #if row['']
        #print(row['drug'], ' op ',  row['DATUM'], 'Eerder gestart')
    l_cur_drugs
    #row['drug'] = new_df['']
print(d_trajectories)
new_df

{'MTX': 'MTX0'}
{'MTX': 'MTX0', 'LEF': 'LEF1'}
{'MTX': 'MTX0', 'LEF': 'LEF1'}
{'LEF': 'LEF1', 'SASP': 'SASP5'}
{'LEF': 'LEF1', 'SASP': 'SASP5', 'MTX': 'MTX6'}
Double Start!!
EARLIER STOP!!
MTX6
{'LEF': 'LEF1', 'SASP': 'SASP5'}
{'MTX0': ['MTX', '2012-05-12', '2012-05-17'], 'LEF1': ['LEF', '2012-05-13', None], 'SASP5': ['SASP', '2012-05-19', '2012-07-21'], 'MTX6': ['MTX', '2012-05-21', '2012-06-23'], 'SASP7': ['SASP', '2012-05-26', None], 'MTX9': ['MTX', '2012-05-21', '2013-06-21']}


,PATNR,drug,DATUM,actie,trigger
0,1,MTX,2012-05-12,Start,
1,1,LEF,2012-05-13,Start,
2,1,MTX,2012-05-17,Stop,
3,1,SASP,2012-05-19,Start,
4,1,MTX,2012-05-21,Start,
5,1,SASP,2012-05-26,Start,CONFLICT-> is al eerder gestart
6,1,MTX,2012-06-23,Stop,
7,1,MTX,2013-06-21,Stop,CONFLICT-> is al eerder geeindigd
8,1,SASP,2012-07-21,Stop,


### Explore Data - Pattern searching

In [5]:
#def extractFeatures(rules):'\\b(\\d+)\\s(mg)\\b'

df_corrected['XANTWOORD'].str.extract('(\S+) (naar|nr) (\S+) (\S+) (\S+) (\S+)').dropna(thresh=1)#[['XANTWOORD']]


/home/tjardo/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  This is separate from the ipykernel package so we can avoid doing imports until


,0,1,2,3,4,5
4,hydroxychloroquine,naar,1,d,200,mg
8,methotrexaat,naar,12.5,(,ook,verzoek
13,prednsiolon,naar,"7,5",mg,/,dg
16,sulfasalazine,naar,2,dd,500,mg
23,verwezen,naar,de,reumaconsulente,voor,toelichtingen
27,stappen,naar,"12,5",mg,/,week
29,methotrexaat,naar,15,mg,sc,^lab
34,opgehoogd,nr,2,dd,1000,mg
35,advies,naar,neuroloog,^methotrexaat,25,mg
41,prednison,naar,30,mg,verlagen,^folder


In [40]:
print(new_df['Operation'].unique())
print(new_df['Strength_nr'].unique())
print(new_df['Strength_unit'].unique())
print(new_df['Freq_unit'].unique())
print(new_df['Route'].unique())

[None 'verlagen' 'stoppen' 'continue' 'afbouwen' 'starten' 'verhogen']
[None '200' '12.5' '40' '500' '80' '12,5' '15' '25' '1000' '50' '20' '17.5'
 '1' '7.5' '10' '0' '120' '75' '2011' '3' '7,5' '5' '2' '8' '6']
[None 'mg' 'g']
[None 'dag' 'maand' 'weken' 'wekelijks' 'wk' 'maandelijks' 'jaar']
[None 'subcutaneous injection' 'intraarticular' 'intramuscular' 'oraal']


### Initialize list of modifiers & targets

- Targets = other words for entities
- Modifiers = terms that influence the meaning of the sentence (like negation terms)

In [2]:
import sys
sys.path.append('../') 
sys.path.append('../modules') 

import modules.pyConTextNLP2.pyConTextNLP.pyConText as pyConText
import modules.pyConTextNLP2.pyConTextNLP.itemData as itemData
from IPython.display import display, HTML
#import EMR_functions as em




modifiers = itemData.get_items(
    'featureExtractionRegex.yml', url=False)
targets = itemData.get_items(
    'targets.yml', url=False)
print(modifiers)
# Maybe use different modifiers/ entities for different steps!!

[literal<<7 mg per week>>; category<<['frequency', 'concentration']>>; re<<(\d+(?:.\d+)?)\s(mg|g|milligram|gram)\s?(?:\S+\s){0,2}(?:voor|vr)?\s?(?:elke|iedere|per|p|\/|a)?\s?(weken|wk|week|wekelijks|dagelijks|dag|dg|d|wkn|maand|mnd|maanden|jr|jaar|jaren)\b>>; rule<<bidirectional>>, literal<<Per week x mg>>; category<<['frequency', 'concentration']>>; re<<(?:elke|iedere|per|\/)?\s?(weken|wk|week|wekelijks|dagelijks|dag|dg|d|wkn|maand|mnd|maanden|jr|jaar|jaren)\s(\d+(?:.\d+)?)\s(mg|g|milligram|microgram|gram)(?:\s|$)>>; rule<<bidirectional>>, literal<<voor x jaar>>; category<<['duration']>>; re<<(?:voor|vr|dan)?\s?(\d+(?:.\d+)?)\s(weken|wk|week|wkn|maand|mnd|maanden|jr|jaar|jaren)\b>>; rule<<bidirectional>>, literal<<2 wkn 20 mg>>; category<<['frequency', 'concentration']>>; re<<(\d+(?:.\d+)?)\s(weken|wk|week|wekelijks|dagelijks|dag|dg|d|wkn|maand|mnd|maanden|jr|jaar|jaren)\s(\d+(?:.\d+)?)\s(mg|g|milligram|gram)(?:\s|$)>>; rule<<bidirectional>>, literal<<1 x / 2 weken>>; category<<['freq

### Markup sentence with the modifiers / targets

In [3]:
def markup_sentence(s, modifiers, targets, prune_inactive=True):
    """
    """
    markup = pyConText.ConTextMarkup()
    markup.setRawText(s)
    markup.cleanText()
    
    markup.markItems(modifiers, mode="modifier")
    markup.markItems(targets, mode="target")
    markup.pruneMarks()
    markup.dropMarks('Exclusion')
    # apply modifiers to any targets within the modifiers scope
    markup.applyModifiers()
    markup.pruneSelfModifyingRelationships()
    if prune_inactive:
        markup.dropInactiveModifiers()
    return markup

## Split the document into sentences and process each sentence

pyConTextNLP comes with a simple sentence splitter in ``helper.py``. I have not been maintaining this and have recently been using TextBlob to split sentences. A known problem with either sentence splitting solution is enumerated lists that don't use periods.

Important: You might need to install the 'punkt' package from nltk

In [4]:
clrs = {\
    "route": "green",
    "target": "blue",
    "definite_negated_existence": "red",
    "continue": "indianred",
    "hypothetical": "orange",
    "probable_existence": "forestgreen",
    "definite_existence": "green",
    "historic": "goldenrod",
    "frequency": "pink",
    "concentration": "green",
    "change": "violet"
}

In [5]:
def fillDictionary(l_capture, l_labels):
    d_features = {'freq_nr' : None, 'freq_unit' : None, 'strength_nr' : None, 'strength_unit': None, 
                  'route': None, 'operation': None, 'continue' : None, 'hypothetical' : None, 'change' : None}
    for x in l_capture:
        d_sent = literal_eval(x)
        d_features.update(d_sent)
    d_features['hypothetical'] = ('hypothetical' in l_labels)
    d_features['continue'] = ('continue' in l_labels)
    d_features['change'] = ('change' in l_labels)
    #print(d_features)
    return d_features


def extractFeatures(report):
    l_entry = []
    counter = 0
    lbls = []
    l_capture = []
    target = 'None'
    for sentence in report.split('^'):
        try:
            context = readContext(sentence)
        except:
            print('--------------------BREAAKK ------------------')
            print(sentence)
            break
        if len(context.getSectionMarkups()) != 0:
            for line in str(context.getSectionMarkups()[0]).split('\n'):
                if 'MODIFIED BY:' in line:
                    lbls.extend(re.findall(r'<category>\s\[\'(.*)\'\]\s</category>', line))
                    l_capture.extend(re.findall(r'<capture>\s(.*)\s</capture>', line))
                elif 'TARGET:' in line:
                    target = re.findall(r'<phrase>\s(.*)\s</phrase>', line)
                    lbls.extend(['probable_existence'])
                elif '********************************' in line or \
                    '__________________________________________' in line:
                    
                    d_features = fillDictionary(l_capture, lbls)
                    if (target) != 'None':
                        #print(d_features)
                        #print(sentence)
                        #print(target)
                        #print(d_features)
                        #display(HTML(html.mark_document_with_html(context ,colors = clrs, default_color="black")))
                        l_entry.append([target[0], d_features['strength_nr'], 
                                        d_features['strength_unit'], 
                                        d_features['freq_nr'], 
                                        d_features['freq_unit'], 
                                        d_features['route'], 
                                        d_features['operation'], 
                                        d_features['continue'],
                                        d_features['hypothetical'],
                                        d_features['change']
                                        ])
                    lbls = []
                    l_capture = []
                    target = 'None'
                    
    if l_entry == []:
        l_entry.append([None]*10)
    return l_entry



In [7]:
import xml.etree.ElementTree
import matplotlib.pyplot as plt
import modules.pyConTextNLP2.pyConTextNLP.display.html as html
from textblob import TextBlob
import modules.pyConTextNLP2
from IPython.display import display, HTML
from ast import literal_eval



def readContext(report):
    context = pyConText.ConTextDocument()
    blob = TextBlob(report.lower())
    count = 0
    rslts = []
    
    for s in blob.sentences:
        m = markup_sentence(s.raw, modifiers=modifiers, targets=targets)
        rslts.append(m)
        
    print(rslts)
    for r in rslts:
        context.addMarkup(r)
    #g = context.getDocumentGraph()
    return context

units = ['7.5 mg per week mtx', 'Per wk 1 dd 15 mg mtx', 'das < 1.6 / mtx verhogen naar 10','verhogen mtx tot 31 a 12', 'azathioprine verhogen naar 2 dd 75 mg', 'plan : depomedrol 80 mg i.v', 'afhankelijk van methotrexaat en hydroxychloroquine door', 'verlaag  mtx', 'verhogen van 20 tot 3.5 g per mnd mtx.', 'nr 2 d 100 mg mtx', 'methotrexaat 2 wk 20 mg',
        'naar 10 mg mtx', '1 x elke week mtx', 'mtx hervatten lagere dosis : 7.5 mg / wk per os ', 'mtx 25 mg s.c. ', 
         'van 20 naar 3.5 mg mtx', 'mtx verhoogd naar 15 mg ', 'mtx verhogen naar 1 x per week', 'mtx 1 x 2 weken', 'mtx verlagen naar 0', 'mtx verlagen naar 1 x per week']

for x in units:
    x = x.replace("mtx", "methotrexaat")
    context = readContext(x)
    #print(context)
    display(HTML(html.mark_document_with_html(context ,colors = clrs, default_color="black")))

#import re

#context =readContext('methotrexaat 20 mg per 2 wk')
#display(HTML(html.mark_document_with_html(context ,colors = clrs, default_color="black")))

#rex = '(?:start)'
#print(re.findall(rex, 'start methotrexaat'))


[__________________________________________
rawText: 7.5 mg per week methotrexaat
cleanedText: 7.5 mg per week methotrexaat
********************************
TARGET: <id> 283228028271198324506854659195669614431 </id> <phrase> methotrexaat </phrase> <category> ['target'] </category> <capture> {} </capture> 
----MODIFIED BY: <id> 283227549733096738350255594190209584991 </id> <phrase> 7.5 mg per week </phrase> <category> ['frequency', 'concentration'] </category> <capture> {'strength_nr': '7.5', 'strength_unit': 'mg', 'freq_unit': 'week'} </capture> 
__________________________________________
]


[__________________________________________
rawText: per wk 1 dd 15 mg methotrexaat
cleanedText: per wk 1 dd 15 mg methotrexaat
********************************
TARGET: <id> 283231018342051612842955439544355295071 </id> <phrase> methotrexaat </phrase> <category> ['target'] </category> <capture> {} </capture> 
__________________________________________
]


[__________________________________________
rawText: das < 1.6 / methotrexaat verhogen naar 10
cleanedText: das < 1.6 / methotrexaat verhogen naar 10
********************************
TARGET: <id> 283235985947841257216922554750041362271 </id> <phrase> methotrexaat </phrase> <category> ['target'] </category> <capture> {} </capture> 
----MODIFIED BY: <id> 283235713402962208147601232958852206431 </id> <phrase> < </phrase> <category> ['hypothetical'] </category> <capture> {'operation': '<'} </capture> 
----MODIFIED BY: <id> 283235395698030525947607482847611359071 </id> <phrase> verhogen </phrase> <category> ['change'] </category> <capture> {'operation': 'verhogen'} </capture> 
----MODIFIED BY: <id> 283235562869453431045359805225346568031 </id> <phrase> naar 10 </phrase> <category> ['dosage'] </category> <capture> {'dosage_nr': '10'} </capture> 
__________________________________________
]


[__________________________________________
rawText: verhogen methotrexaat tot 31 a 12
cleanedText: verhogen methotrexaat tot 31 a 12
********************************
TARGET: <id> 283240513837328947423816025786803064671 </id> <phrase> methotrexaat </phrase> <category> ['target'] </category> <capture> {} </capture> 
----MODIFIED BY: <id> 283240225446817395501627185286823841631 </id> <phrase> tot 31  </phrase> <category> ['dosage'] </category> <capture> {'dosage_nr': '31'} </capture> 
__________________________________________
]


[__________________________________________
rawText: azathioprine verhogen naar 2 dd 75 mg
cleanedText: azathioprine verhogen naar 2 dd 75 mg
********************************
TARGET: <id> 283245672382990251174836741433409441631 </id> <phrase> azathioprine </phrase> <category> ['target'] </category> <capture> {} </capture> 
----MODIFIED BY: <id> 283245080548616269620234917660100431711 </id> <phrase> verhogen </phrase> <category> ['change'] </category> <capture> {'operation': 'verhogen'} </capture> 
----MODIFIED BY: <id> 283244930807389117660636865862034296671 </id> <phrase> naar 2 dd 75 mg </phrase> <category> ['frequency', 'concentration'] </category> <capture> {'freq_nr': '2', 'freq_unit': 'd', 'strength_nr': '75', 'strength_unit': 'mg'} </capture> 
__________________________________________
]


...

[__________________________________________
rawText: methotrexaat verhoogd naar 15 mg
cleanedText: methotrexaat verhoogd naar 15 mg
********************************
TARGET: <id> 283307747648320177283341283118497697631 </id> <phrase> methotrexaat </phrase> <category> ['target'] </category> <capture> {} </capture> 
----MODIFIED BY: <id> 283307232665263834565146925082820513631 </id> <phrase> verhoogd </phrase> <category> ['change'] </category> <capture> {'operation': 'verhoogd'} </capture> 
----MODIFIED BY: <id> 283307480649412504212523592875385065311 </id> <phrase> naar 15 mg </phrase> <category> ['concentration'] </category> <capture> {'strength_nr': '15', 'strength_unit': 'mg'} </capture> 
__________________________________________
]


[__________________________________________
rawText: methotrexaat verhogen naar 1 x per week
cleanedText: methotrexaat verhogen naar 1 x per week
********************************
TARGET: <id> 283312475985059028579008865821453750111 </id> <phrase> methotrexaat </phrase> <category> ['target'] </category> <capture> {} </capture> 
----MODIFIED BY: <id> 283311847705730290462811749017927585631 </id> <phrase> verhogen naar 1 x per week </phrase> <category> ['change'] </category> <capture> {'operation': 'verhogen', 'strength_nr': '1', 'strength_unit': None, 'freq_unit': 'week'} </capture> 
__________________________________________
]


[__________________________________________
rawText: methotrexaat 1 x 2 weken
cleanedText: methotrexaat 1 x 2 weken
********************************
TARGET: <id> 283316423132115489228307776181059489631 </id> <phrase> methotrexaat </phrase> <category> ['target'] </category> <capture> {} </capture> 
----MODIFIED BY: <id> 283316026199021292763976432525868306271 </id> <phrase> 1 x 2 weken </phrase> <category> ['dosage', 'frequency'] </category> <capture> {'dosage_nr': '1', 'freq_nr': '2', 'freq_unit': 'weken'} </capture> 
__________________________________________
]


[__________________________________________
rawText: methotrexaat verlagen naar 0
cleanedText: methotrexaat verlagen naar 0
********************************
TARGET: <id> 283320423362040834434712874215111954271 </id> <phrase> methotrexaat </phrase> <category> ['target'] </category> <capture> {} </capture> 
----MODIFIED BY: <id> 283319940862531122564896929532454408031 </id> <phrase> verlagen naar 0 </phrase> <category> ['change'] </category> <capture> {'operation': 'verlagen', 'strength_nr': '0', 'strength_unit': None} </capture> 
__________________________________________
]


[__________________________________________
rawText: methotrexaat verlagen naar 1 x per week
cleanedText: methotrexaat verlagen naar 1 x per week
********************************
TARGET: <id> 283325103369600552029134524856258301791 </id> <phrase> methotrexaat </phrase> <category> ['target'] </category> <capture> {} </capture> 
----MODIFIED BY: <id> 283324552733871077891988249725803466591 </id> <phrase> verlagen naar 1 x per week </phrase> <category> ['change'] </category> <capture> {'operation': 'verlagen', 'strength_nr': '1', 'strength_unit': None, 'freq_unit': 'week'} </capture> 
__________________________________________
]


In [7]:

new_df = pd.DataFrame(columns=['XANTWOORD', 'Target', 'Strength_nr', 'Strength_unit', 'Freq_nr', 'Freq_unit', 'Route', 'Operation', 'Continue', 'Hypothetical', 'Change'])
#df_corrected['XANTWOORD'].sample(frac=0.2, random_state=777).head(10).apply(lambda x: extractFeatures(str(x))).tolist()
#print(new_df)

for index, row in df_corrected.head(500).iterrows():
    l_entry = extractFeatures(row['XANTWOORD'])
    for entry in l_entry: 
        # print('ent', entry)
        new_df = new_df.append({'ID' : row['id'], 'XANTWOORD' : row['XANTWOORD'], 'Target' : entry[0], 'Strength_nr': entry[1], 'Strength_unit': entry[2], 'Freq_nr' : entry[3], 'Freq_unit': entry[4], 'Route': entry[5], 'Operation': entry[6], 'Continue': entry[7], 'Hypothetical': entry[8], 'Change': entry[9]}, ignore_index=True)
new_df



NameError: name 'df_corrected' is not defined

## Build the Pipeline

In [9]:

route_map = {"i.v.": "intravenous", "iv": "intravenous", "i.v": "intravenous", "i.a." : "intraarticular", "ia" : "intraarticular", "i.a" : "intraarticular", \
             "sc" : "subcutaneous injection", "s.c." : "subcutaneous injection", "s.c" : "subcutaneous injection", "per os": "oraal", \
             "p.o." : "oraal", "p.o" : "oraal", "po" : "oraal", "i.m.": "intramuscular", "im": "intramuscular", "i.m": "intramuscular"}
freq_map = {"wkn": "weken", "wk": "wekelijks", "week": "wekelijks", "dagelijks" : "dag", "dg" : "dag", "d" : "dag", \
            "mnd" : "maand", "maanden": "maandelijks", "jr" : "jaar", "jaren" : "jaarlijks" }
operation_map = {"verlaag" : "verlagen", "stop" : "stoppen", "gecontinueerd" : "continue", "doorgaan" : "continue",  
        "afbouw" : "afbouwen", "start" : "starten", "verhoogd" : "verhogen", "verlaagd" : "verlagen", \
        "verhoog" : "verhogen", "ophogen": "verhogen", "opgehoogd" : "verhogen", "verlaag" : "verlagen", \
        "vervolgen": "continue", "vervolgd": "continue", "door" : "continue", "continueren": "continue", \
        "vervolgd": "continue", "gecontinueerd": "continue", \
                "gestopt" : "stoppen", "gestart": "starten", "stopt" : "stoppen"}

new_df.replace({"Route": route_map}, inplace=True)
new_df.replace({"Freq_unit": freq_map}, inplace=True)
new_df.replace({"Operation": operation_map}, inplace=True)

new_df

,XANTWOORD,Target,Strength_nr,Strength_unit,Freq_nr,Freq_unit,Route,Operation,Continue,Hypothetical,Change,ID
0,iom kloppenburg ^ - nu niet alarmerend voor sp...,None,None,None,None,None,None,None,None,None,None,22396.0
1,co begin juli met foto s ^nu expect ; bij mee...,None,None,None,None,None,None,None,None,None,None,30214.0
2,- morgen langskomen op leeg plekje in mijn sp...,None,None,None,None,None,None,None,None,None,None,15832.0
3,pt is aangemeld voor de raz ^lab nog laten afn...,None,None,None,None,None,None,None,None,None,None,43472.0
4,` verlaag hydroxychloroquine naar 1 d 200 mg ...,hydroxychloroquine,200,mg,1,dag,None,verlagen,False,False,True,20045.0
5,` verlaag hydroxychloroquine naar 1 d 200 mg ...,hydroxychloroquine,None,None,6,maand,None,stoppen,False,True,True,20045.0
6,- med zo door ^ - co 6 mnd,None,None,None,None,None,None,None,None,None,None,31653.0
7,adalimumab en leflunomide door . ^lf vandaag h...,adalimumab,None,None,None,None,None,continue,True,False,False,26176.0
8,adalimumab en leflunomide door . ^lf vandaag h...,leflunomide,None,None,None,None,None,continue,True,False,False,26176.0
9,"co volg week , goed drinken",None,None,None,None,None,None,None,None,None,None,7195.0


In [22]:
new_df[new_df['Freq_unit'] == '4 weken']['XANTWOORD']
#new_df['Operation'].unique()
#new_df['Freq_nr'].unique()
new_df.loc[504]['XANTWOORD']

'iom mkl : ^ - diclofenac 2 d 75 mg ter verlichting klachten ^ - over 3 - 4 weken nog eenmaal lab gezien methotrexaat per 1 - 7 gestart is , gb = gb ^ - polico volgt in nov '

In [17]:
context = readContext(new_df.loc[39]['XANTWOORD'])
display(HTML(html.mark_document_with_html(context ,colors = clrs, default_color="black")))

In [10]:
new_df.to_csv(r'DF_annotated_featureExtract.csv', sep='|', index=True)

In [51]:
# Medication Trajectory 


'lala mtx 25 mg .'

In [11]:

locf_df = pd.DataFrame(columns=['ID', 'Med', 'Datum', 'Conf'])
values1 = [['MTX',34, 0.5], ['LEF', 41, 0.4], ['RTX', 31, 0.6]]
for x in range(len(values1)):
    locf_df = locf_df.append({'ID' : x, 'Med' : values1[x][0], 'Datum' : values1[x][1], 'Conf': values1[x][2]}, ignore_index=True)
mv_df = pd.DataFrame(columns=['ID', 'Med', 'Datum', 'Conf'])
values2 = [['RTX',30, 0.9], ['KD', 21, 0.7], ['LAX', 64, 0.6]]
for x in range(len(values2)):
    mv_df = mv_df.append({'ID' : x, 'Med' : values2[x][0], 'Datum' : values2[x][1], 'Conf': values2[x][2]}, ignore_index=True)

def leastDistance(d_candidates):
    for key, value in d_candidates.items():
        print(value[0])
    print(min(d_candidates, key=d_candidates.get[0]))
    return
        
    
    
best_candidates = {'LOCF': [locf_df['Datum'].min(), locf_df['Datum'].idxmin()], 'MV': [mv_df['Datum'].min(), mv_df['Datum'].idxmin()]}
leastDistance(best_candidates)

31
21


In [ ]:
locf_df = pd.DataFrame(columns=['ID', 'Med', 'Datum', 'Conf'])
values1 = [['MTX',31, 0.5], ['LEF', 41, 0.4], ['RTX', 32, 0.6]]
for x in range(len(values1)):
    new_df = new_df.append({'ID' : x, 'Med' : values[0], 'Datum' : values[1], 'Conf': values[2]}, ignore_index=True)

locf_df